[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/experimental/bert-search-test/bertcomparison.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/experimental/bert-search-test/bertcomparison.ipynb)

# BERT search in Pinecone

## **Dependencies**


In [ ]:
!pip install --quiet pandas
!pip install --quiet progressbar2

In [ ]:
import re
import bz2
import time
import numpy
import pandas as pd
from typing import List
from statistics import mean
from progressbar import progressbar

## **Dataset**

The dataset used in this notebook is the dbpedia dataset that contains full abstracts of Wikipedia articles, usually the first section.


Downloading the dataset

In [ ]:
!rm long_abstracts_en.ttl.bz2
!wget http://downloads.dbpedia.org/2016-10/core-i18n/en/long_abstracts_en.ttl.bz2

We will be conducting a similar test as described in this blog post: [Speeding up BERT Search in Elasticsearch](https://towardsdatascience.com/speeding-up-bert-search-in-elasticsearch-750f1f34f455#e7c4-d62eca28921b). The code is avaliable on this link: https://github.com/DmitryKey/bert-solr-search.git.

**Parsing the dataset**


We will be using the same method that was used for parsing the dataset in the blogpost. Original source of this method can be found on this link: https://github.com/DmitryKey/bert-solr-search/blob/master/src/data_utils.py

In [ ]:
def parse_dbpedia_data(source_file, max_docs: int):
    """
    Parses the input file of abstracts and returns an iterable
    :param max_docs: maximum number of input documents to process; -1 for no limit
    :param source_file: input file
    :return: yields document by document to the consumer
    """
    global VERBOSE
    count = 0
    max_tokens = 0

    if -1 < max_docs < 50:
        VERBOSE = True

    percent = 0.1
    bulk_size = (percent / 100) * max_docs

    print(f"bulk_size={bulk_size}")

    if bulk_size <= 0:
        bulk_size = 1000

    for line in source_file:
        line = line.decode("utf-8")

        # skip commented out lines
        comment_regex = '^#'
        if re.search(comment_regex, line):
            continue

        token_size = len(line.split())
        if token_size > max_tokens:
            max_tokens = token_size

        # skip lines with 20 tokens or less, because they tend to contain noise
        # (this may vary in your dataset)
        if token_size <= 20:
            continue

        first_url_regex = '^<([^\>]+)>\s*'

        x = re.search(first_url_regex, line)
        if x:
            url = x.group(1)
            # also remove the url from the string
            line = re.sub(first_url_regex, '', line)
        else:
            url = ''

        # remove the second url from the string: we don't need to capture it, because it is repetitive across
        # all abstracts
        second_url_regex = '^<[^\>]+>\s*'
        line = re.sub(second_url_regex, '', line)

        # remove some strange line ending, that occurs in many abstracts
        language_at_ending_regex = '@en \.\n$'
        line = re.sub(language_at_ending_regex, '', line)

        # form the input object for this abstract
        doc = {
            "_text_": line,
            "url": url,
            "id": count+1
        }

        yield doc
        count += 1

        if count % bulk_size == 0:
            print(f"Processed {count} documents", end="\r")

        if count == max_docs:
            break

    source_file.close()
    print("Maximum tokens observed per abstract: {}".format(max_tokens))

If you are experiencing an issue with RAM, lower the number of MAX_DOCS.

In [ ]:
MAX_DOCS = 1000000

source_file = bz2.BZ2File("long_abstracts_en.ttl.bz2", "r")
docs_iter = parse_dbpedia_data(source_file, MAX_DOCS)

**Creating a pandas dataframe**

In [ ]:
id = []
text = []

for doc in docs_iter:
    id.append(doc['id'])
    text.append(doc['_text_'])

data = pd.DataFrame({'id': id, 'text': text})

In [ ]:
data.head()

**Generating embeddings using BERT**

Generating embeddings is a time consuming process. Please use GPU or lower the number of MAX_DOCS. On Google Colab you should be expecting around 1.5 hours for 1M documents with GPU.

In [ ]:
!pip install --quiet sentence_transformers==1.0.4
!pip install --quiet tqdm==4.41.1

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
# # expensive: downloads the model, creates embeddings
import os
import h5py

#If embeddings not present, run inferencing and store vectors as h5 file
if not os.path.exists('embeddings.h5'):
    model = SentenceTransformer('bert-base-nli-mean-tokens')
    sentence_embeddings = model.encode(text, show_progress_bar=True)
    file = h5py.File("embeddings.h5", "w")
    file.create_dataset('bert', data=sentence_embeddings)
    file.close()
    
#If embedding file present, load the vectors directly
hf = h5py.File('embeddings.h5', 'r')
vec_embeds = list(hf.get('bert'))

#Add embeddings to DataFrame
data['embeddings'] = pd.Series(vec_embeds)

## **Pinecone**

In [ ]:
!pip install --quiet -U pinecone-client

In [ ]:
import pinecone

In [ ]:
# load Pinecone API key
api_key = 'YOUR_API_KEY'

pinecone.init(api_key=api_key)

index_name = 'bert-stats-test'


[Get the Pinecone API key](https://www.pinecone.io/start/) if you don’t have one already.

In [ ]:
items_to_upload = data[['id', 'embeddings']]
items_to_upload = [tuple(x) for x in items_to_upload.to_numpy()]

We are defining a variable which we will be using to query vectors in batches. The reason for this is to make our results comparable to the ones published in the blog. By querying in batches and then dividing the elapsed time with the same number in the end, we minimize the influence of the networking time.

In [ ]:
def upload_items(items_to_upload: List, batch_size: int) -> float:
    print(f"\nUpserting {len(items_to_upload)} vectors...")
    start = time.perf_counter()
    upsert_cursor = index.upsert(items=items_to_upload,batch_size=batch_size)
    end = time.perf_counter()
    return (end - start) / 60.0 # minutes

def restart_service(index_name: str, shards: int, timeout: int = 300):
    if index_name in pinecone.list_indexes():
        pinecone.delete_index(index_name)
    pinecone.create_index(index_name,metric='cosine', shards=shards)
    index = pinecone.Index(index_name)
    return index

def query(test_items: List, index):
    print(f"\nQuerying...")
    times = []
    #For single queries, we pick 10 queries 
    for test_item in test_items[:10]:
        start = time.perf_counter()
        #test_item is an array of [id,vector]
        query_results = index.query(queries=[test_item[1]],disable_progress_bar=True)              # querying vectors with top_k=10
        end = time.perf_counter()
        times.append((end-start))           
    #For batch queries, we pick 100 vectors at perform 10 queries
    print(f"\n Batch Querying...")
    batch_times = []
    for i in range(0,10000, 1000):
        start = time.perf_counter() 
        batch_items = test_items[i:i+1000]
        vecs = [item[1] for item in batch_items]
        query_results = index.query(queries=vecs,disable_progress_bar=True)
        end = time.perf_counter()
        batch_times.append((end-start))
    return mean(times)*1000,mean(batch_times)*1000


Testing uploading and querying

In [ ]:
BATCH_SIZE = 1000
NUMBER_OF_DOCS = [10000, 100000, 200000, 400000, 600000, 800000, 1000000]


upsert_times = {}                  
query_times = {}
batch_query_times = {}
for doc_size in progressbar(NUMBER_OF_DOCS):
    if doc_size > len(items_to_upload):
        print(f"There are no {doc_size} vectors to be uploaded.")
        continue
    test_vectors = items_to_upload[:10000]
    index = restart_service(index_name, shards=3)
    time_for_upsert = upload_items(items_to_upload[:doc_size], BATCH_SIZE)
    time_for_query,time_for_batch_query = query(test_vectors, index)
    upsert_times[doc_size] = time_for_upsert
    query_times[doc_size] = time_for_query
    batch_query_times[doc_size] = time_for_batch_query

## **Displaying results**



In [ ]:
time_results = pd.DataFrame({
    'number_of_docs': upsert_times.keys(),
    'indexing_time(min)': upsert_times.values(),
    'avg_search_speed(ms)': query_times.values(),
    'avg_batch_search_speed(ms), batch_size=1000':batch_query_times.values()
})
time_results['index_size(mb)'] = (time_results['number_of_docs'] * len(items_to_upload[0][1]) * 32) / 8000000 # megabytes

In [ ]:
time_results

In [ ]:
#For batch querying, sending 100 queries at once
print(query_times)
print(batch_query_times)

In [ ]:
time_results.plot(x="number_of_docs", y=["indexing_time(min)"], kind="bar")

In [ ]:
time_results.plot(x="number_of_docs", y=["avg_search_speed(ms)"], kind="bar")

In [ ]:
time_results.plot(x="number_of_docs", y=["avg_batch_search_speed(ms), batch_size=1000"], kind="bar")

## Estimating Search Speed Without Network Overhead 

It is difficult to look at the real speed of Pinecone's engine when we are querying a deployment across the cloud which included overheads like network, parsing, authentication etc. These overheads contribute a lot to the observed speeds rather than search speed itself.

One way to estimate the speed of the engine is to look at the returned search speeds as :

overhead + num_queries\*search_speed_per_query 

The overhead will change with every call but it generally does not fluctuate too much,so for rough estimates we can assume this as a constant in our equation.

For 1 million documents and assuming the overhead is constant: 

single query
overhead + 1\*search_speed_per_query = 35.68ms
batched query
overhead + 1000\*search_speed_per_query = 7020.4ms

Solving for search_speed_per_query

999\*search_speed_per_query = 7020.4-35.68

search_speed_per_query = 6.98 ms

Using this logic let's look at estimated search speed per query on Pinecone's engine

In [ ]:
speed_per_query = {}
for num_docs in query_times.keys():
    batch_speed = batch_query_times[num_docs]
    speed = query_times[num_docs]
    speed_per_query[num_docs] = (batch_speed-speed)/999

In [ ]:
time_results['speed_per_query'] = speed_per_query.values()
time_results.plot(x="number_of_docs", y=["speed_per_query"], kind="bar")
speed_per_query

## Calculating Recall

It's important that while performing ANN search we maintain a good recall while speeding up search.
We will calculate the rank-k@k recall by taking results from KNN (exact search) to be the ground truth.


In [ ]:
#Create an exact index and upload items
exact_index_name = 'exactsearch'
# exact_index = pinecone.create_index(name=exact_index_name,metric='cosine',shards=3,engine_type='exact')
exact_index = pinecone.Index(exact_index_name)
upsert_cursor = exact_index.upsert(items=items_to_upload[:doc_size],batch_size=1000)

Once we have uploaded the same items on the exact index as well, we will queries both exact and approximate indexes to compare the results.


In [ ]:
import concurrent.futures
index = pinecone.Index(name=index_name)
NUM_TEST_QUERIES = 10
with concurrent.futures.ThreadPoolExecutor() as executor: 
    approx_res = executor.map(lambda i: index.unary_query( test_vectors[i][1], top_k=100), range(NUM_TEST_QUERIES))  
    
exact_index = pinecone.Index(name=exact_index_name)
with concurrent.futures.ThreadPoolExecutor() as executor:     
    exact_res = executor.map(lambda i: exact_index.unary_query( test_vectors[i][1], top_k=100), range(NUM_TEST_QUERIES))  
    

In [ ]:
import numpy as np
def anns_recall(r_exact, r):
    assert(len(r_exact.scores) == len(r.scores))
    exact_rank_k_score = r_exact.scores[-1]
    indicator = [s >= exact_rank_k_score for s in r.scores]
    return sum(indicator) / len(indicator)


def approx_loss(r_exact, r):
    return np.quantile([ abs(ext_s - apprx_s) for ext_s, apprx_s in zip(r_exact.scores, r.scores)], 0.5)


recalls = []
a_loss = []
for exact_r, r in zip(exact_res, approx_res):
    recalls.append( anns_recall(exact_r, r) )
    a_loss.append(approx_loss(exact_r, r))

print("Accuracy results over 10 test queries:")
print(f"The average recall @rank-k is {sum(recalls)/len(recalls)}")
print(f"The median approximation loss is {np.quantile(a_loss, 0.5)}")

Delete the indexes, once done.

In [ ]:
pinecone.delete_index(index_name)
pinecone.delete_index(exact_index_name)